In [10]:
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torch.utils.data import random_split

import matplotlib.pyplot as plt

In [7]:
def fn(x):
    return ((2*x[:, 0].unsqueeze(1)) + 3) + (3 * x[:, 1].unsqueeze(1))

In [11]:
def noise(shape, low=-1.0, high=1.0):
    return (high - low) * torch.rand(shape) + low

def fn(x):
    # return np.sin(x) + noise(x.shape, low=0, high=1)
    # return np.sin(x) #+ noise(x.shape, low=0, high=1)
    # return x*10 #+ noise(x.shape, low=0, high=1)

    # return torch.sin(x[:, 0].unsqueeze(1)) + torch.cos(x[:, 1].unsqueeze(1)) + noise(x.shape, low=0, high=1) # x[:, 0] are xs; x[:, 0] are the ys
    return ((2*x[:, 0].unsqueeze(1)) + 3) + (3 * x[:, 1].unsqueeze(1))

class x_y_ds(Dataset):
    def __init__(self, fn, N):
        # self.xs = torch.linspace(min, max, N).unsqueeze(1)
        # self.ys = fn(self.xs)

        self.xs = torch.rand(N, 2)
        self.ys = fn(self.xs)

    def __len__(self):
        return len(self.ys)

    def __getitem__(self, idx):
        return self.xs[idx], self.ys[idx]
    
data = x_y_ds(fn, N = 100)
training_dataset, test_dataset = random_split(
    data, [80, 20]
)
    
train_dataloader = DataLoader(dataset = training_dataset, batch_size=10, shuffle=True)
test_dataloader = DataLoader(dataset = test_dataset, batch_size=10, shuffle=False)

In [103]:
def loss_fn(y_hat, y):
    return (y_hat - y)**2

def loss_grad(y_hat, y):
    return 2 * (y_hat - y)

In [145]:
lr = 0.01
ws = torch.tensor([0.5, 0.6])
b = torch.tensor([0.5])

epochs = 100
for epoch in range(epochs):
    epoch_loss_train = 0
    num_batches = 0
    for x_batch, y_batch in train_dataloader:
        bs = y_batch.shape[0]
        y_hat = (x_batch @ ws).unsqueeze(1) + b # prediction
        losses_train = loss_fn(y_hat, y_batch).mean() # loss
        epoch_loss_train += losses_train
        # updation
        delta = loss_grad(y_hat, y_batch)
        ws -= lr * (x_batch.T @ delta).squeeze() / bs
        b -= lr * delta.mean()

        num_batches += 1

    epoch_loss_train /= num_batches

    # testing
    epoch_loss_test = 0
    num_batches = 0
    for x_batch, y_batch in test_dataloader:
        bs = y_batch.shape[0]
        y_hat = (x_batch @ ws).unsqueeze(1) + b # prediction
        losses_test = loss_fn(y_hat, y_batch).mean() # loss
        epoch_loss_test += losses_test
        num_batches += 1
    epoch_loss_test /= num_batches

    if epoch % 5 == 0:
            print(f"Epoch {epoch} | train loss = {epoch_loss_train:.4f} | test loss = {epoch_loss_test:.4f}")
        

Epoch 0 | train loss = 16.8647 | test loss = 12.0301
Epoch 5 | train loss = 1.4253 | test loss = 1.0281
Epoch 10 | train loss = 0.1536 | test loss = 0.1209
Epoch 15 | train loss = 0.0463 | test loss = 0.0437
Epoch 20 | train loss = 0.0342 | test loss = 0.0343
Epoch 25 | train loss = 0.0302 | test loss = 0.0308
Epoch 30 | train loss = 0.0272 | test loss = 0.0280
Epoch 35 | train loss = 0.0246 | test loss = 0.0255
Epoch 40 | train loss = 0.0223 | test loss = 0.0232
Epoch 45 | train loss = 0.0201 | test loss = 0.0212
Epoch 50 | train loss = 0.0182 | test loss = 0.0193
Epoch 55 | train loss = 0.0164 | test loss = 0.0176
Epoch 60 | train loss = 0.0148 | test loss = 0.0160
Epoch 65 | train loss = 0.0134 | test loss = 0.0146
Epoch 70 | train loss = 0.0121 | test loss = 0.0133
Epoch 75 | train loss = 0.0110 | test loss = 0.0121
Epoch 80 | train loss = 0.0100 | test loss = 0.0111
Epoch 85 | train loss = 0.0090 | test loss = 0.0101
Epoch 90 | train loss = 0.0081 | test loss = 0.0092
Epoch 95 | t